In [1]:
import pandas as pd
import numpy as np
from ngcc_ml import data_tools
from itertools import product

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# ML
import tensorflow as tf

import keras
from keras import Model
from keras import regularizers
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout

In [2]:
def process_data(df_in, oneHotCols=[], scaledCols=[]):
    df = df_in.copy()
    scaler = StandardScaler()
    oneHotEncoder = OneHotEncoder(categories = "auto", sparse = False)
    if scaledCols != []:
        df[scaledCols] = scaler.fit_transform(df[scaledCols])
    #if oneHotCols != []:
    #    df[encoded_columns] = oneHotEncoder.fit_transform(alldata[oneHotCols])
    return df


def group_split_evaluate(model, df_xy, feature_cols, target_col="BindingEnergy", n_splits=10, group_col="catalyst_name"):
    all_scores, models = [], []
    split_groups = GroupShuffleSplit(test_size=1.0/n_splits, n_splits=n_splits, random_state = 7).split(df_xy, groups=df_xy[group_col])
    model.save_weights("initial_weights.h5")
    for train_inds, test_inds in split_groups:
        model.load_weights("initial_weights.h5")
        train = df_xy.iloc[train_inds]
        test = df_xy.iloc[test_inds]
        X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
        model.fit(X_train_group, y_train_group, epochs=150, batch_size=16)
        mae = model.evaluate(X_test_group, y_test_group)
        all_scores.append(mae)
        models.append(model)
        print('MAE on test set: {:.2f}'.format(mae))
    print("mean:", np.mean(all_scores))
    return all_scores


def create_ffnn_regression(inp_dim, all_lsize, l2_reg=0.0001, drop_val = 0.1):
    model = Sequential()
    model.add(Dense(all_lsize[0], input_dim=inp_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg)))
    model.add(Dropout(drop_val))
    for lsize in all_lsize[1:]:
        model.add(Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(Dropout(drop_val))
    model.add(Dense(1, kernel_initializer="normal"))
    return model


def sample_list(inlist):
    return inlist[np.random.randint(len(inlist))]


def sample_nested_list(inlist):
    sampled = []
    for sublist in inlist:
        sampled.append(sample_list(sublist))
    return sampled


def hyperparam_sample(df_xy, feature_cols, lsize_opts, l2_reg_opts, drop_val_opts, n_splits=5, hyperparam_samples=6):
    
    lsize_list, l2_reg_list, drop_val_list, group_scores_list = [], [], [], []
    for i in range(hyperparam_samples):
        lsize_sample = sample_nested_list(lsize_opts)
        l2_reg_sample = sample_list(l2_reg_opts)
        drop_val_sample = sample_list(drop_val_opts)   
        model = create_ffnn_regression(len(feature_cols), lsize_sample, l2_reg_sample, drop_val_sample)
        model.compile(loss="mean_absolute_error", optimizer="adam")
        group_scores = group_split_evaluate(model, df_xy, feature_cols, n_splits=n_splits)

        lsize_list.append(lsize_sample)
        l2_reg_list.append(l2_reg_sample)
        drop_val_list.append(drop_val_sample)
        group_scores_list.append(group_scores)
    
    df_results = pd.DataFrame({"lsize":lsize_list, "l2_reg":l2_reg_list, "drop_val":drop_val_list, "scores":group_scores_list})
    return df_results


def hyperparam_grid(df_xy, feature_cols, lsize_opts, l2_reg, drop_val, target_col="BindingEnergy", n_splits=5):
    
    lsize_list, l2_reg_list, drop_val_list, group_scores_list = [], [], [], []
    for layer in product(*lsize_opts):
        lsize = sample_nested_list(lsize_opts)   
        model = create_ffnn_regression(len(feature_cols), lsize, l2_reg, drop_val)
        model.compile(loss="mean_absolute_error", optimizer="adam")
        group_scores = group_split_evaluate(model, df_xy, feature_cols, target_col=target_col, n_splits=n_splits)

        lsize_list.append(lsize)
        l2_reg_list.append(l2_reg)
        drop_val_list.append(drop_val)
        group_scores_list.append(group_scores)
    
    df_results = pd.DataFrame({"lsize":lsize_list, "l2_reg":l2_reg_list, "drop_val":drop_val_list, "scores":group_scores_list})
    return df_results

In [3]:
df = pd.read_csv("/home/nricke/work/ngcc_ml/DidItBindv5.csv", index_col=0)
df.rename(columns={"Catalyst Name":"catalyst_name"}, inplace=True)
df["Doesitbind"] = df["Doesitbind"].astype("int")
#df = df[df["Doesitbind"] == 1]

In [4]:
feature_cols = {"SpinDensity", "ChElPGNeutralCharge", "ChargeDifference", "IonizationEnergy", "OrthoOrPara", "Meta", "FartherThanPara", "DistanceToN", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "NeighborSpinDensity", 'NeighborChElPGCharge', 'NeighborChargeDifference', "AromaticExtent", "RingEdge", "NumNitrogens", "NumHeteroatoms", "charge", "atom_plane_deviation", "ring_plane_deviation", "ring_nitrogens"}
not_scaled_cols = {"OrthoOrPara", "Meta", "FartherThanPara", "NumberOfHydrogens", "IsInRingSize6", "IsInRingSize5", "RingEdge", "NumNitrogens", "NumHeteroatoms", "ring_nitrogens", "charge"}

In [22]:
target_cols = ["Doesitbind", "BindingEnergy"]

df_active = df[df["Doesitbind"] == 1]
df_spec = df.copy()
df_spec.loc[df_spec['Doesitbind'] == 0, 'BindingEnergy'] = 0.0
X_active = df_scale[feature_cols]
y_active = df_scale[["Doesitbind", "BindingEnergy"]]

In [6]:
all_lsize = [1024, 512, 256, 128, 16]
drop_val = 0.1
l2_reg = 0.00001
inp_dim = len(feature_cols)

In [7]:
target_cols = ["BindingEnergy"]

visible = Input(shape=inp_dim)
hidden = Dense(all_lsize[0], activation="relu", kernel_initializer="normal", kernel_regularizer=keras.regularizers.l2(l2_reg))(visible)
hidden = Dropout(drop_val)(hidden)
for lsize in all_lsize[1:]:
    hidden = Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg))(hidden)
    hidden = Dropout(drop_val)(hidden)
be_reg = Dense(1, name="BindingEnergy", kernel_initializer="normal")(hidden)
#dib_class = Dense(1, name="Doesitbind", activation="sigmoid")(hidden)
model = Model(inputs=visible, outputs=[be_reg])
model.compile(loss={
                    #"Doesitbind": "binary_crossentropy", 
                    "BindingEnergy":"mean_absolute_error"}, 
              optimizer="adam",
              metrics={"BindingEnergy":"mean_absolute_error"},
             )

evv = []
split_groups = GroupKFold(n_splits=10).split(df_active[feature_cols], df_active[target_cols], df_active["catalyst_name"])
model.save_weights("initial_weights.h5")
for train_inds, test_inds in split_groups:
    model.load_weights("initial_weights.h5")
    train = df_active.iloc[train_inds]
    test = df_active.iloc[test_inds]
    X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
    model.fit(X_train_group, {"BindingEnergy": y_train_group["BindingEnergy"]}, 
              epochs=100, batch_size=16)
    evv.append(model.evaluate(X_test_group, {"BindingEnergy": y_test_group["BindingEnergy"]}))

Epoch 1/100
42/42 [==============================] - 0s 3ms/step - loss: 0.2092 - mean_absolute_error: 0.1917
Epoch 2/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1321 - mean_absolute_error: 0.1158
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1277 - mean_absolute_error: 0.1125
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1278 - mean_absolute_error: 0.1134
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1120 - mean_absolute_error: 0.0984
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1079 - mean_absolute_error: 0.0950
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1018 - mean_absolute_error: 0.0894
Epoch 8/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0995 - mean_absolute_error: 0.0876
Epoch 9/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1019 - mean_absolute_error: 0.0904
Epoch 10/1

42/42 [==============================] - 0s 3ms/step - loss: 0.0584 - mean_absolute_error: 0.0525
Epoch 75/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0542 - mean_absolute_error: 0.0484
Epoch 76/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0569 - mean_absolute_error: 0.0511
Epoch 77/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0546 - mean_absolute_error: 0.0488
Epoch 78/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0575 - mean_absolute_error: 0.0518
Epoch 79/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0578 - mean_absolute_error: 0.0520
Epoch 80/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0556 - mean_absolute_error: 0.0498
Epoch 81/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0536 - mean_absolute_error: 0.0479
Epoch 82/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0548 - mean_absolute_error: 0.0491
Epoch 83/100
4

42/42 [==============================] - 0s 3ms/step - loss: 0.0622 - mean_absolute_error: 0.0564
Epoch 48/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0639 - mean_absolute_error: 0.0582
Epoch 49/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0613 - mean_absolute_error: 0.0556
Epoch 50/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0633 - mean_absolute_error: 0.0576
Epoch 51/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0624 - mean_absolute_error: 0.0567
Epoch 52/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0615 - mean_absolute_error: 0.0559
Epoch 53/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0605 - mean_absolute_error: 0.0549
Epoch 54/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0618 - mean_absolute_error: 0.0563
Epoch 55/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0595 - mean_absolute_error: 0.0540
Epoch 56/100
4

42/42 [==============================] - 0s 3ms/step - loss: 0.0803 - mean_absolute_error: 0.0727
Epoch 21/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0801 - mean_absolute_error: 0.0726
Epoch 22/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0871 - mean_absolute_error: 0.0797
Epoch 23/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0776 - mean_absolute_error: 0.0703
Epoch 24/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0768 - mean_absolute_error: 0.0696
Epoch 25/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0761 - mean_absolute_error: 0.0691
Epoch 26/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0773 - mean_absolute_error: 0.0704
Epoch 27/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0767 - mean_absolute_error: 0.0699
Epoch 28/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0746 - mean_absolute_error: 0.0679
Epoch 29/100
4

42/42 [==============================] - 0s 3ms/step - loss: 0.0534 - mean_absolute_error: 0.0483
Epoch 94/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0552 - mean_absolute_error: 0.0502
Epoch 95/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0509 - mean_absolute_error: 0.0458
Epoch 96/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0510 - mean_absolute_error: 0.0459
Epoch 97/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0480 - mean_absolute_error: 0.0429
Epoch 98/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0494 - mean_absolute_error: 0.0443
Epoch 99/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0506 - mean_absolute_error: 0.0456
Epoch 100/100
3/3 [==============================] - 0s 1ms/step - loss: 0.0756 - mean_absolute_error: 0.0706
Epoch 1/100
42/42 [==============================] - 0s 3ms/step - loss: 0.2198 - mean_absolute_error: 0.2022
Epoch 2/100
42/4

...

42/42 [==============================] - 0s 4ms/step - loss: 0.0587 - mean_absolute_error: 0.0534
Epoch 59/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0593 - mean_absolute_error: 0.0540
Epoch 60/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0588 - mean_absolute_error: 0.0535
Epoch 61/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0611 - mean_absolute_error: 0.0559
Epoch 62/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0584 - mean_absolute_error: 0.0533
Epoch 63/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0593 - mean_absolute_error: 0.0542
Epoch 64/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0577 - mean_absolute_error: 0.0525
Epoch 65/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0629 - mean_absolute_error: 0.0578
Epoch 66/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0599 - mean_absolute_error: 0.0548
Epoch 67/100
4

42/42 [==============================] - 0s 3ms/step - loss: 0.0703 - mean_absolute_error: 0.0639
Epoch 32/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0680 - mean_absolute_error: 0.0617
Epoch 33/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0680 - mean_absolute_error: 0.0617
Epoch 34/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0683 - mean_absolute_error: 0.0621
Epoch 35/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0705 - mean_absolute_error: 0.0643
Epoch 36/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0681 - mean_absolute_error: 0.0620
Epoch 37/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0694 - mean_absolute_error: 0.0634
Epoch 38/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0632 - mean_absolute_error: 0.0572
Epoch 39/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0655 - mean_absolute_error: 0.0595
Epoch 40/100
4

42/42 [==============================] - 0s 3ms/step - loss: 0.1160 - mean_absolute_error: 0.1027
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1135 - mean_absolute_error: 0.1010
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1060 - mean_absolute_error: 0.0942
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0999 - mean_absolute_error: 0.0887
Epoch 8/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1081 - mean_absolute_error: 0.0974
Epoch 9/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1103 - mean_absolute_error: 0.1000
Epoch 10/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1029 - mean_absolute_error: 0.0930
Epoch 11/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0937 - mean_absolute_error: 0.0841
Epoch 12/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0948 - mean_absolute_error: 0.0856
Epoch 13/100
42/42 

Epoch 78/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0559 - mean_absolute_error: 0.0508
Epoch 79/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0531 - mean_absolute_error: 0.0480
Epoch 80/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0519 - mean_absolute_error: 0.0468
Epoch 81/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0541 - mean_absolute_error: 0.0490
Epoch 82/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0524 - mean_absolute_error: 0.0473
Epoch 83/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0517 - mean_absolute_error: 0.0467
Epoch 84/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0505 - mean_absolute_error: 0.0455
Epoch 85/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0520 - mean_absolute_error: 0.0470
Epoch 86/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0542 - mean_absolute_error: 0.0492
E

42/42 [==============================] - 0s 3ms/step - loss: 0.0576 - mean_absolute_error: 0.0518
Epoch 52/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0620 - mean_absolute_error: 0.0563
Epoch 53/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0588 - mean_absolute_error: 0.0531
Epoch 54/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0573 - mean_absolute_error: 0.0517
Epoch 55/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0553 - mean_absolute_error: 0.0497
Epoch 56/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0550 - mean_absolute_error: 0.0494
Epoch 57/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0583 - mean_absolute_error: 0.0527
Epoch 58/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0573 - mean_absolute_error: 0.0517
Epoch 59/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0589 - mean_absolute_error: 0.0533
Epoch 60/100
4

In [8]:
evv

[[0.07501772046089172, 0.06952400505542755],
 [0.11487628519535065, 0.11011987179517746],
 [0.0755666196346283, 0.07055259495973587],
 [0.07925621420145035, 0.0742621198296547],
 [0.0713500827550888, 0.06648404151201248],
 [0.08183380216360092, 0.0767388790845871],
 [0.05766788125038147, 0.05273420363664627],
 [0.08212106674909592, 0.07725744694471359],
 [0.09548591822385788, 0.09055209904909134],
 [0.09349530190229416, 0.08839522302150726]]

In [10]:
evv = np.array(evv)
np.mean(evv[:,1])

0.07766204848885536

In [50]:
from keras import backend as K

In [36]:
y_true = [1]
y_pred = [1]
mse = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.SUM)
mse(y_true, y_pred).numpy()

0

In [52]:
mask_value = 0

def masked_loss_function(y_true, y_pred):
    mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
    #return K.binary_crossentropy(y_true * mask, y_pred * mask)
    #return K.mean(K.square((y_pred-y_true) * mask))
    masked_square_error = K.square((y_pred-y_true) * mask)

    
# https://github.com/keras-team/keras/issues/7065
def masked_mse(mask_value):
    def f(y_true, y_pred):
        mask_true = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
        return masked_mse
    f.__name__ = 'Masked MSE (mask_value={})'.format(mask_value)
    return f

masked_mse0 = masked_mse(0)


In [53]:
target_cols = ["Doesitbind", "BindingEnergy"]

visible = Input(shape=inp_dim)
hidden = Dense(all_lsize[0], activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(visible)
hidden = Dropout(drop_val)(hidden)
for lsize in all_lsize[1:]:
    hidden = Dense(lsize, activation='relu', kernel_initializer="normal", kernel_regularizer=regularizers.l2(l2_reg))(hidden)
    hidden = Dropout(drop_val)(hidden)
be_reg = Dense(1, name="BindingEnergy", kernel_initializer="normal")(hidden)
dib_class = Dense(1, name="Doesitbind", activation="sigmoid")(hidden)
model = Model(inputs=visible, outputs=[dib_class, be_reg])
model.compile(loss={
                    "Doesitbind": "binary_crossentropy",
                    "BindingEnergy": masked_mse0,
                    #"BindingEnergy":"mean_absolute_error"
                    }, 
              optimizer="adam",
              metrics={"Doesitbind":"accuracy", "BindingEnergy":"mean_absolute_error"},
             )
    
evv = []
split_groups = GroupKFold(n_splits=3).split(df_spec[feature_cols], df_spec[target_cols], df_spec["catalyst_name"])
model.save_weights("initial_weights.h5")
for train_inds, test_inds in split_groups:
    model.load_weights("initial_weights.h5")
    train = df_spec.iloc[train_inds]
    test = df_spec.iloc[test_inds]
    X_train_group, y_train_group, X_test_group, y_test_group = data_tools.scale_data(train, test, feature_cols, target_cols)
    model.fit(X_train_group, {"Doesitbind": y_train_group["Doesitbind"], "BindingEnergy": y_train_group["BindingEnergy"]}, 
              epochs=100, batch_size=16)
    evv.append(model.evaluate(X_test_group, {"Doesitbind": y_test_group["Doesitbind"], "BindingEnergy": y_test_group["BindingEnergy"]}))

Epoch 1/100


ValueError: in user code:

    /home/nricke/local/opt/miniconda/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/nricke/local/opt/miniconda/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/nricke/local/opt/miniconda/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/nricke/local/opt/miniconda/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/nricke/local/opt/miniconda/envs/tfgpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_los

In [26]:
np.array(evv)

array([[2.94431057e-02, 5.36475682e-06, 2.75438521e-02, 1.70673072e-01,
        2.75438521e-02],
       [2.03121938e-02, 6.74322109e-06, 1.84144210e-02, 1.89903840e-01,
        1.84144210e-02],
       [3.65920253e-02, 4.57927513e-07, 3.45254168e-02, 1.68269232e-01,
        3.45254168e-02],
       [2.42464039e-02, 2.66014126e-06, 2.22904943e-02, 1.82926834e-01,
        2.22904943e-02],
       [2.44491994e-02, 6.32990341e-05, 2.25344673e-02, 1.68292686e-01,
        2.25344673e-02],
       [2.62779687e-02, 7.28500845e-06, 2.44954769e-02, 1.68292686e-01,
        2.44954769e-02],
       [2.60231011e-02, 9.03424097e-06, 2.37995349e-02, 1.77884609e-01,
        2.37995349e-02],
       [3.20157893e-02, 4.72087231e-06, 2.98807900e-02, 1.75480768e-01,
        2.98807900e-02],
       [2.59635951e-02, 9.52231767e-06, 2.39305533e-02, 1.92307696e-01,
        2.39305533e-02],
       [2.67791282e-02, 4.46531658e-06, 2.48795729e-02, 1.90361440e-01,
        2.48795729e-02]])

In [27]:
model.evaluate(X_test_group, {"Doesitbind": y_test_group["Doesitbind"], "BindingEnergy": y_test_group["BindingEnergy"]})

5/5 [==============================] - 0s 2ms/step - loss: 0.1522 - Doesitbind_loss: 0.0017 - BindingEnergy_loss: 0.1472


[0.15223973989486694, 0.0016895182197913527, 0.14724817872047424]

In [31]:
model.predict(X_test_group)

[array([[1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9989256 ],
        [0.9993717 ],
        [0.9996958 ],
        [1.        ],
        [1.        ],
        [0.99936444],
        [1.        ],
        [1.        ],
        [0.9990896 ],
        [0.99889886],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9980259 ],
        [1.        ],
        [0.9999931 ],
        [0.9999449 ],
        [1.        ],
        [1.        ],
        [0.9999995 ],
        [1.        ],
        [1.        ],
        [0.99957126],
        [0.9979144 ],
        [1.        ],
        [0.99820566],
        [1.        ],
        [0.9999083 ],
        [0.9999062 ],
        [0.9999442 ],
        [1.        ],
        [0.99984956],
        [0.9988034 ],
        [0.99988544],
        [0.99998164],
        [1.        ],
        [1.        ],
        [1

In [45]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 22)]              0         
_________________________________________________________________
dense_33 (Dense)             (None, 1024)              23552     
_________________________________________________________________
dropout_29 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_31 (Dropout)         (None, 256)               0   

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test_group, y_pred)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.annotate("R$^2$ = 0.83", xy=(-1,-0.1))
ax.annotate("10-fold CV MAE = 0.080 eV", xy=(-1,-0.145))
plt.plot([-1.1,0],[-1.1,0.], color="black")
plt.ylim([-1.1,0])
plt.xlim([-1.1,0])
plt.ylabel(r"O$_2$ Binding Energy (eV) Predicted")
plt.xlabel(r"O$_2$ Binding Energy (eV) DFT")
plt.scatter(y_test_group, y_pred, s=150)
#plt.show()
plt.savefig("Regression_R2_MAE.png")

In [ ]:
model.fit(X_train_group, y_train_group, epochs=200, batch_size=16, class_weight={0:4, 1:1})

In [ ]:
_, accuracy = model.evaluate(X_test_group, y_test_group)
print('Accuracy: %.2f' % (accuracy*100))